In [1]:
import tensorflow as tf
import keras
from tensorflow.python.framework.ops import disable_eager_execution
import numpy as np
import os,sys
import h5py
import argparse,socket
import datetime
%load_ext tensorboard

# Clear any logs from previous runs
#!rm -rf ./logs/

In [2]:
print("tensorflow version:",tf.__version__)
print("keras version:",keras.__version__)
print("python version:",sys.version)

tensorflow version: 2.6.0
keras version: 2.6.0
python version: 3.8.0 (default, Jul 23 2021, 11:57:41) 
[GCC 9.3.0]


In [3]:
tf.debugging.set_log_device_placement(False)
#disable_eager_execution()
tf.executing_eagerly()

True

In [4]:
print(tf.config.list_physical_devices(device_type=None))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Creates virtual GPU with 1GB memory limit
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=20000)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
        print(e)
print(logical_gpus) #AVX stands for Advanced Vector Extensions

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPU, 1 Logical GPUs
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2021-10-26 13:46:55.208993: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-26 13:46:55.611180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20000 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:41:00.0, compute capability: 8.6


# globals

In [5]:
# working directories and paths
BASE_DIR = os.getcwd()
sys.path.append(BASE_DIR)
print("BASE_DIR:",BASE_DIR)

#global parameters
MAX_NUM_POINT = 4096
BATCH_SIZE = 32
MAX_EPOCH = 100
BASE_LEARNING_RATE =0.001
OPTIMIZER = 'adam'
MOMENTUM = 0.9
DECAY_STEP = 300000
DECAY_RATE = 0.5
NUM_CLASSES = 13

HOSTNAME = socket.gethostname()


BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

BASE_DIR: /home/liteandfog/pointnet/sem_seg


# logging

In [6]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/gradient_tape/' + current_time + '/train'
writer = tf.summary.create_file_writer(logdir)

# data

In [7]:
def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

In [8]:
# Load ALL data
ALL_FILES = ['data/'+line.rstrip() for line in open(os.path.join(BASE_DIR, 'data/indoor3d_sem_seg_hdf5_data/all_files.txt'))]
room_filelist = ['data/'+line.rstrip() for line in open('data/indoor3d_sem_seg_hdf5_data/room_filelist.txt')]
data_batch_list = []
label_batch_list = []
for h5_filename in ALL_FILES:
    data_batch, label_batch = load_h5(h5_filename)
    data_batch_list.append(data_batch)
    label_batch_list.append(label_batch)

In [9]:
data_batches = np.concatenate(data_batch_list, 0)
label_batches = np.concatenate(label_batch_list, 0)
print(data_batches.shape)
print(label_batches.shape)

(23585, 4096, 9)
(23585, 4096)


In [10]:
test_area = 'Area_'+str(6)
train_idxs = []
test_idxs = []
for i,room_name in enumerate(room_filelist):
    if test_area in room_name:
        test_idxs.append(i)
    else:
        train_idxs.append(i)

In [11]:
train_data = data_batches[train_idxs,...]
train_label = label_batches[train_idxs]
test_data = data_batches[test_idxs,...]
test_label = label_batches[test_idxs]
print("training data:",train_data.shape, train_label.shape)
print("test data:",test_data.shape, test_label.shape)
print(f"test/train ratio:{test_data.shape[0]/train_data.shape[0]}")

training data: (20291, 4096, 9) (20291, 4096)
test data: (3294, 4096, 9) (3294, 4096)
test/train ratio:0.16233798235670988


#  weights

In [12]:
Weights_shapes=[[1,9,1,64], [1,1,64,64], [1,1,64,64],[1,1,64,128],[1,1,128,1024],[1024,256],[256,128]]

Weights_shapes_1=[[1,1,1152,512],[1,1,512,256],[1,1,256,13]]


In [13]:
def _kernel(shape):
    initializer = keras.initializers.GlorotNormal() #Xavier
    with tf.device('/cpu:0'): #save on cpu memory
        kernel=tf.Variable(name="weights",initial_value=initializer(shape=shape))
    return kernel

def _biases(output_channels):
    intializer = keras.initializers.Constant(0.0)
    with tf.device('/cpu:0'): #save on cpu memory
        biases=tf.Variable(name="biases",shape=output_channels,initial_value=intializer(shape=[output_channels]))
    return biases

def _bn(momentum):
    with tf.device('/cpu:0'): #save on cpu memory

        bn_init=keras.layers.BatchNormalization(momentum=momentum,
           beta_initializer='zeros', gamma_initializer='ones',
           moving_mean_initializer='ones', moving_variance_initializer='ones')
    return bn_init

# layers

In [14]:
class Dense(keras.layers.Layer):
    def __init__(self, shape, name=None):
        super().__init__(name=name)
        self.shape=shape
        
    def build(self,input_shape):
        self.w = _kernel(self.shape)
        self.b = _biases(self.shape[-1])
        self.bn = _bn(BN_INIT_DECAY)
        
    def call(self, x, bn=True, activation=True, is_training=True):
        y = tf.matmul(x, self.w) + self.b
        
        if bn:
            y = self.bn(y,training = is_training)
        
        if activation:
            y = tf.nn.relu(y)
        
        return y

class Conv2d(keras.layers.Layer):
    def __init__(self, shape, strides=[1,1,1,1], padding='VALID', data_format='NHWC', name=None):
        super().__init__(name=name)
        self.shape=shape
        self.strides = strides
        self.padding = padding
        self.data_format = data_format
        
    def build(self, input_shape):
        self.filters = _kernel(self.shape)
        self.bias = _biases(self.shape[-1])
        self.bn = tf.keras.layers.BatchNormalization(momentum=0.5,
        beta_initializer='zeros', gamma_initializer='ones',
        moving_mean_initializer='ones', moving_variance_initializer='ones')
        
    def call(self, inputs, bn=True, activation=True, is_training=True):
        y = tf.nn.conv2d(inputs, filters=self.filters, strides=self.strides, padding=self.padding , data_format=self.data_format)
        y = tf.nn.bias_add(y, self.bias)
        
        if bn:
            y = self.bn(y,training = is_training)
        
        if activation:
            y = tf.nn.relu(y)
        return y
    
class MaxPool(keras.layers.Layer):
    def __init__(self, batch_size=32 ,name=None):
        super().__init__(name=name)
        self.batch_size=batch_size
        
    def call(self, pcd):
        y = tf.nn.max_pool(pcd, ksize=[1, MAX_NUM_POINT, 1, 1], strides=[1, 2, 2, 1], padding='VALID')
        y = tf.reshape(y, [self.batch_size, -1])
        return y
    
class TransformMatrix(keras.layers.Layer):
    def __init__(self, shape, K, batch_size=32, name=None):
        super().__init__(name=name)
        self.shape=shape
        self.batch_size=batch_size
        self.K=K
        
    def build(self, input_shape):
        self.w = _kernel(self.shape)
        bias = _biases(self.shape[-1]).assign_add(tf.constant(np.eye(self.K).flatten(), dtype=tf.float32))
        self.b = bias
        
    def call(self, pcd):
        
        y = tf.matmul(pcd, self.w)
        y = tf.nn.bias_add(y, self.b)
        y = tf.reshape(y, [self.batch_size, self.K, self.K])
        return y #BxKxK 

# model

In [15]:
class PointNet(keras.Model):

    def __init__(self):
        
        super(PointNet, self).__init__()
        self.conv1_1 = Conv2d(Weights_shapes[0])
        self.conv1_2 = Conv2d(Weights_shapes[1])  
        self.conv1_3 = Conv2d(Weights_shapes[2])
        self.conv1_4 = Conv2d(Weights_shapes[3])
        self.conv1_5 = Conv2d(Weights_shapes[4])
        
        self.max_pool = MaxPool()
        self.fc_1 = Dense(Weights_shapes[5])
        self.fc_2 = Dense(Weights_shapes[6])
        
        
        self.conv2_1 = Conv2d(Weights_shapes_1[0])
        self.conv2_2 = Conv2d(Weights_shapes_1[1])  
        self.conv2_3 = Conv2d(Weights_shapes_1[2])
        
        self.dropout=tf.keras.layers.Dropout(rate=0.7)
        
      
    def call(self, inputs, is_training=True):
        
        batch_size = inputs.shape[0]
        num_point = inputs.shape[1]
        
        net = tf.expand_dims(inputs, -1)
        net = self.conv1_1(net, is_training=is_training)
        net = self.conv1_2(net, is_training=is_training)
        net = self.conv1_3(net, is_training=is_training)
        net = self.conv1_4(net, is_training=is_training)
        
        points_feat1 = self.conv1_5(net, is_training=is_training) 
        
        # MAX
        pc_feat1 = self.max_pool(points_feat1)
        
        # FC
        pc_feat1 = tf.reshape(pc_feat1, [batch_size, -1])
        pc_feat1 = self.fc_1(pc_feat1,is_training=is_training)
        pc_feat1 = self.fc_2(pc_feat1,is_training=is_training)
        #print(pc_feat1)
           
        # CONCAT 
        pc_feat1_expand = tf.tile(tf.reshape(pc_feat1, [batch_size, 1, 1, -1]), [1, num_point, 1, 1])
        points_feat1_concat = tf.concat(axis=3, values=[points_feat1, pc_feat1_expand])    
        
        #print(points_feat1_concat.shape)
        # CONV 
        net = self.conv2_1(points_feat1_concat,is_training=is_training)
        net = self.conv2_2(net,is_training=is_training)
        net = self.dropout(net,training=is_training)
        net = self.conv2_3(net,is_training=is_training)
        net = tf.squeeze(net, [2])
        
        return net

# initialize the model

In [16]:
batch = train_data[:BATCH_SIZE]
label = tf.cast(train_label[:BATCH_SIZE],'int64')
model = PointNet()
model(batch)

2021-10-26 13:47:16.895935: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8200
2021-10-26 13:47:18.438853: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


<tf.Tensor: shape=(32, 4096, 13), dtype=float32, numpy=
array([[[0.40291032, 0.        , 0.        , ..., 0.        ,
         0.        , 0.43335667],
        [1.2509133 , 0.19500989, 1.6917155 , ..., 0.        ,
         1.48139   , 0.        ],
        [0.        , 0.3572611 , 0.        , ..., 0.7992417 ,
         1.7681752 , 0.8204681 ],
        ...,
        [0.        , 0.2614817 , 1.6911188 , ..., 0.        ,
         0.        , 0.5358056 ],
        [0.22568426, 0.        , 1.484625  , ..., 0.        ,
         0.        , 0.        ],
        [0.8759454 , 0.6394088 , 1.0660714 , ..., 0.        ,
         0.38041803, 0.        ]],

       [[0.        , 0.0492514 , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 1.8799969 , 0.        , ..., 0.88067824,
         0.4265388 , 1.222658  ],
        [0.24277559, 1.2993528 , 0.        , ..., 0.        ,
         0.04485658, 1.7135996 ],
        ...,
        [0.2189036 , 0.        , 0.30088982, ..., 0.

In [17]:
model.summary()

Model: "point_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2d)              multiple                  896       
_________________________________________________________________
conv2d_1 (Conv2d)            multiple                  4416      
_________________________________________________________________
conv2d_2 (Conv2d)            multiple                  4416      
_________________________________________________________________
conv2d_3 (Conv2d)            multiple                  8832      
_________________________________________________________________
conv2d_4 (Conv2d)            multiple                  136192    
_________________________________________________________________
max_pool (MaxPool)           multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  26

# loss

In [18]:
def get_loss(pred, label):
    """ pred: B,N,13
        label: B,N """
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=label)
    return tf.reduce_mean(loss)

# train

In [19]:
get_loss(model(batch), label)

<tf.Tensor: shape=(), dtype=float32, numpy=2.7196634>

In [20]:
class ExponentialDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    """A LearningRateSchedule that uses an exponential decay schedule.
    Returns:
      A 1-arg callable learning rate schedule that takes the current optimizer
      step and outputs the decayed learning rate, a scalar `Tensor` of the same
      type as `initial_learning_rate`.
      modification: clips the learning rate !
    """

    def __init__(self,initial_learning_rate,decay_steps,decay_rate,staircase=False,name=None):
        """Applies exponential decay to the learning rate.
        Args:
          initial_learning_rate: A scalar `float32` or `float64` `Tensor` or a
            Python number.  The initial learning rate.
          decay_steps: A scalar `int32` or `int64` `Tensor` or a Python number.
            Must be positive.  See the decay computation above.
          decay_rate: A scalar `float32` or `float64` `Tensor` or a
            Python number.  The decay rate.
          staircase: Boolean.  If `True` decay the learning rate at discrete
            intervals
          name: String.  Optional name of the operation.  Defaults to
            'ExponentialDecay'.
        """
        super(ExponentialDecay, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.staircase = staircase
        self.name = name

    def __call__(self, step):
        step=step*BATCH_SIZE
        with tf.name_scope(self.name or "ExponentialDecay") as name:
            initial_learning_rate = tf.convert_to_tensor(
                                 self.initial_learning_rate, name="initial_learning_rate")
            dtype = initial_learning_rate.dtype
            decay_steps = tf.cast(self.decay_steps, dtype)
            decay_rate = tf.cast(self.decay_rate, dtype)
            global_step_recomp = tf.cast(step, dtype)
            p = global_step_recomp / decay_steps
            
            if self.staircase:
                p = tf.floor(p)
            
            return tf.maximum(tf.multiply(initial_learning_rate, tf.pow(decay_rate, p), name=name), 0.00001)
        

In [21]:
def shuffle_data(data, labels):
    """ Shuffle data and labels.
        Input:
          data: B,N,... numpy array
          label: B,... numpy array
        Return:
          shuffled data, label and shuffle indices
    """
    idx = np.arange(len(labels))
    np.random.shuffle(idx)
    return data[idx, ...], labels[idx], idx

In [22]:
def train(epoch_num=MAX_EPOCH):
    
    is_training = True
    
    # learning rate
    lr = ExponentialDecay(BASE_LEARNING_RATE, DECAY_STEP, DECAY_RATE, staircase=True)
    optimizer = tf.optimizers.Adam(lr)
    
    
    for epoch in range(epoch_num):
        print(f"LOG: EPOCH NUMBER: {epoch}")
        train_one_epoch(epoch, optimizer, is_training)
        eval_mean_loss, eval_acc = eval_one_epoch()
        with writer.as_default():
            tf.summary.scalar('eval_accuracy',eval_acc , step=epoch)
        
        
    # Save the model
    model.save_weights('./model.h5') 

In [23]:
def train_one_epoch(epoch, optimizer, is_training=True):
    
    
    current_data, current_label, _ = shuffle_data(train_data[:,0:MAX_NUM_POINT,:], train_label)
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    for batch_idx in range(num_batches):
        if batch_idx % 100 == 0:
            print(f'Current batch/total batch num: {batch_idx}/{num_batches}')
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        
        batch = current_data[start_idx:end_idx, :, :]
        label = tf.cast(current_label[start_idx:end_idx],'int64')
        
        pred,loss = train_step(model, batch, label  , optimizer, is_training=is_training)
        pred = np.argmax(pred, 2)
        correct = np.sum(pred == label)
        total_correct += correct
        total_seen += (BATCH_SIZE*MAX_NUM_POINT)
        loss_sum += loss
    
    with writer.as_default():
        tf.summary.scalar('loss', loss_sum/num_batches, step=epoch)
        tf.summary.scalar('accuracy',total_correct/total_seen , step=epoch)
    
    print(f'mean loss: {loss_sum}/{float(num_batches)}')
    print(f'accuracy: {total_correct/total_seen}')

              
def train_step(model, inputs , labels, optimizer, is_training=True):
    
    #single model iteration for tensorBoard graph's plot.
    
    with tf.GradientTape() as tape:
        pred = model(inputs, is_training=is_training)
        loss = get_loss(pred, labels)
        
    grads = tape.gradient(loss , model.trainable_weights)
    
    optimizer.apply_gradients(
    (grad, var) 
    for (grad, var) in zip(grads, model.trainable_variables) 
    if grad is not None)
    
    
    return pred,loss 

# evaluation

In [24]:
def eval_one_epoch():
    """ ops: dict mapping from string to tf ops """
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    print('----')
    current_data = test_data[:,0:MAX_NUM_POINT,:]
    current_label = tf.cast(np.squeeze(test_label),'int64')
    
    num_batches = current_data.shape[0] // BATCH_SIZE
    
    for batch_idx in range(num_batches):
        print(batch_idx,num_batches)
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        batch = current_data[start_idx:end_idx, :, :]
        label = tf.cast(current_label[start_idx:end_idx],'int64')
        
        pred = model(batch,is_training)
        loss = get_loss(pred, label)
        
        pred = np.argmax(pred, 2)
        

        correct = np.sum(pred == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += (BATCH_SIZE*MAX_NUM_POINT)
        loss_sum += (loss*BATCH_SIZE)
        
        
        eval_mean_loss = loss_sum / float(total_seen/MAX_NUM_POINT)
        eval_accuracy = total_correct / float(total_seen)
        
        print(f'eval mean loss: {eval_mean_loss}')
        print(f'eval accuracy: {eval_accuracy}')
        
        return eval_mean_loss, eval_accuracy

In [25]:
%tensorboard --logdir $logdir

In [ ]:
train()

LOG: EPOCH NUMBER: 0
Current batch/total batch num: 0/634
Current batch/total batch num: 100/634
Current batch/total batch num: 200/634
Current batch/total batch num: 300/634
Current batch/total batch num: 400/634
Current batch/total batch num: 500/634
Current batch/total batch num: 600/634
mean loss: 757.0279541015625/634.0
accuracy: 0.7645443456030042
----
0 102
eval mean loss: 0.6077287197113037
eval accuracy: 0.91339111328125
LOG: EPOCH NUMBER: 1
Current batch/total batch num: 0/634
Current batch/total batch num: 100/634
Current batch/total batch num: 200/634


In [ ]:
eval_one_epoch()